<a href="https://colab.research.google.com/github/nuel071003/JST-Tugas/blob/main/JST_14.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import keras
import math
import numpy as np
from keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator


In [ ]:
# Removed conflicting import: from keras.preprocessing.image import ImageDataGenerator
from keras.layers import BatchNormalization
from keras.layers import Conv2D, Dense, Input, add, Activation, AveragePooling2D, GlobalAveragePooling2D, concatenate
from keras.initializers import he_normal
from keras.callbacks import LearningRateScheduler, TensorBoard, ModelCheckpoint
from keras.models import Model
from keras import optimizers, regularizers
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
# Hyperparameters
growth_rate = 12
depth = 100
compression = 0.5

img_rows, img_cols = 32, 32
img_channels = 3
num_classes = 10
batch_size = 64
epochs = 10  # Reduced for faster training in this demonstration
iterations = 782
weight_decay = 1e-4

mean = [125.307, 122.95, 113.865]
std = [62.9932, 62.0887, 66.7048]

def scheduler(epoch):
    if epoch < 150:
        return 0.1
    if epoch < 225:
        return 0.01
    return 0.001

In [ ]:
def densenet(img_input, classes_num):

    def conv(x, out_filters, k_size):
        return Conv2D(filters=out_filters,
                      kernel_size=k_size,
                      strides=(1,1),
                      padding='same',
                      kernel_initializer='he_normal',
                      kernel_regularizer=regularizers.l2(weight_decay),
                      use_bias=False)(x)

    def dense_layer(x):
        return Dense(units=classes_num,
                     activation='softmax',
                     kernel_initializer='he_normal',
                     kernel_regularizer=regularizers.l2(weight_decay))(x)

    def bn_relu(x):
        x = BatchNormalization(momentum=0.9, epsilon=1e-5)(x)
        x = Activation('relu')(x)
        return x

    def bottleneck(x):
        channels = growth_rate * 4
        x = bn_relu(x)
        x = conv(x, channels, (1,1))
        x = bn_relu(x)
        x = conv(x, growth_rate, (3,3))
        return x

    def transition(x, inchannels):
        outchannels = int(inchannels * compression)
        x = bn_relu(x)
        x = conv(x, outchannels, (1,1))
        x = AveragePooling2D((2,2), strides=(2, 2))(x)
        return x, outchannels

    def dense_block(x, blocks, nchannels):
        concat = x
        for i in range(blocks):
            x = bottleneck(concat)
            concat = concatenate([x,concat], axis=-1)
            nchannels += growth_rate
        return concat, nchannels

    nblocks = (depth - 4) // 6
    nchannels = growth_rate * 2

    x = conv(img_input, nchannels, (3,3))
    x, nchannels = dense_block(x, nblocks, nchannels)
    x, nchannels = transition(x, nchannels)
    x, nchannels = dense_block(x, nblocks, nchannels)
    x, nchannels = transition(x, nchannels)
    x, nchannels = dense_block(x, nblocks, nchannels)
    x = bn_relu(x)
    x = GlobalAveragePooling2D()(x)
    x = dense_layer(x)
    return x

In [ ]:
if __name__ == '__main__':
    # Load CIFAR-10 data
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    y_train = keras.utils.to_categorical(y_train, num_classes)
    y_test = keras.utils.to_categorical(y_test, num_classes)
    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')

In [ ]:
    # Normalize data
    for i in range(3):
        x_train[:,:,:,i] = (x_train[:,:,:,i] - mean[i]) / std[i]
        x_test[:,:,:,i] = (x_test[:,:,:,i] - mean[i]) / std[i]

In [ ]:
# Build DenseNet model
img_input = Input(shape=(img_rows, img_cols, img_channels)) #Removed extra space here
output = densenet(img_input, num_classes)
model = Model(img_input, output)

print(model.summary())

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 32, 32, 3)      │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 32, 32, 24)     │            648 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization       │ (None, 32, 32, 24)     │             96 │ conv2d[0][0]           │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation (Activation)   │ (None, 32, 32, 24)     │              0 │ batch_normalization[0… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 32, 32, 48)     │          1,152 │ activation[0][0]       │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_1     │ (None, 32, 32, 48)     │            192 │ conv2d_1[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_1 (Activation) │ (None, 32, 32, 48)     │              0 │ batch_normalization_1… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 32, 32, 12)     │          5,184 │ activation_1[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 32, 32, 36)     │              0 │ conv2d_2[0][0],        │
│                           │                        │                │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_2     │ (None, 32, 32, 36)     │            144 │ concatenate[0][0]      │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_2 (Activation) │ (None, 32, 32, 36)     │              0 │ batch_normalization_2… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 32, 32, 48)     │          1,728 │ activation_2[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_3     │ (None, 32, 32, 48)     │            192 │ conv2d_3[0][0]         │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ activation_3 (Activation) │ (None, 32, 32, 48)     │              0 │ batch_normalization_3… │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 32, 32, 12)     │          5,184 │ activation_3[0][0]     │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 32, 32, 48)     │              0 │ conv2d_4[0][0],        │
│ (Concatenate)             │                        │                │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ batch_normalization_4

 Total params: 793,150 (3.03 MB)

 Trainable params: 769,162 (2.93 MB)

 Non-trainable params: 23,988 (93.70 KB)

None


In [ ]:
# Compile model
sgd = optimizers.SGD(learning_rate=0.1, momentum=0.9, nesterov=True) # Changed 'lr' to 'learning_rate'
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
# Callbacks
tb_cb = TensorBoard(log_dir='./densenet/', histogram_freq=0)
change_lr = LearningRateScheduler(scheduler)
ckpt = ModelCheckpoint('./ckpt.keras', save_best_only=False, mode='auto', save_freq='epoch')
cbks = [change_lr, tb_cb, ckpt]

In [ ]:
# Data augmentation
datagen = ImageDataGenerator(horizontal_flip=True,
                             width_shift_range=0.125,
                             height_shift_range=0.125,
                             fill_mode='constant',
                             cval=0.)
datagen.fit(x_train)

In [32]:
# Train model
model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          steps_per_epoch=iterations,
          epochs=epochs,
          callbacks=cbks,
          validation_data=(x_test, y_test))

Epoch 1/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 7694s 10s/step - accuracy: 0.6709 - loss: 1.4430 - val_accuracy: 0.6847 - val_loss: 1.4616 - learning_rate: 0.1000
Epoch 2/10
782/782 ━━━━━━━━━━━━━━━━━━━━ 324s 415ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6847 - val_loss: 1.4616 - learning_rate: 0.1000
Epoch 3/10
206/782 ━━━━━━━━━━━━━━━━━━━━ 1:31:13 10s/step - accuracy: 0.7361 - loss: 1.2136

KeyboardInterrupt: 

In [33]:
    # Save model
    model.save('densenet.h5')

In [34]:
    # Evaluate model
    print("Evaluating model...")
    y_pred = model.predict(x_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = np.argmax(y_test, axis=1)

Evaluating model...
313/313 ━━━━━━━━━━━━━━━━━━━━ 281s 884ms/step


In [35]:
    # Classification report
    print("Classification Report:")
    print(classification_report(y_true, y_pred_classes, target_names=[str(i) for i in range(num_classes)]))

Classification Report:
              precision    recall  f1-score   support

           0       0.83      0.70      0.76      1000
           1       0.99      0.57      0.73      1000
           2       0.67      0.55      0.60      1000
           3       0.53      0.57      0.55      1000
           4       0.73      0.68      0.71      1000
           5       0.95      0.20      0.33      1000
           6       0.56      0.89      0.69      1000
           7       0.59      0.92      0.71      1000
           8       0.81      0.88      0.85      1000
           9       0.69      0.90      0.78      1000

    accuracy                           0.69     10000
   macro avg       0.73      0.69      0.67     10000
weighted avg       0.73      0.69      0.67     10000



In [36]:
    # Confusion matrix
    print("Confusion Matrix:")
    print(confusion_matrix(y_true, y_pred_classes))

Confusion Matrix:
[[696   1  95  18  26   0  18  35  76  35]
 [ 19 574   1   2   2   0  22  17  52 311]
 [ 39   1 548  35  93   4 186  81  10   3]
 [  9   0  48 568  51   7 175 107  22  13]
 [  9   1  16  31 684   0 135 119   4   1]
 [  1   0  64 323  40 197 144 218  11   2]
 [  7   0  22  44  12   0 892  20   3   0]
 [  7   0  11  23  22   0  14 915   3   5]
 [ 35   3   9   7   6   0  12  10 881  37]
 [ 13   2   2  13   3   0   5  39  22 901]]
